# 1  生成可交互式地图

In [ ]:
# import 相关库
import geemap, os, ee

# 如果后续GEE授权认证时出现问题，则需设置代理地址，并根据本机地址进行调整
# 如果没有问题，可以跳过下段代码
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'

In [ ]:
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

## 1.1  生成基础地图

In [ ]:
Map = geemap.Map()
Map

## 1.2  更改/添加底层地图：Map.add_basemap

In [ ]:
Map.add_basemap('HYBRID')

In [ ]:
Map.basemap_demo()

# 2  交互式工具

## 2.1  inspector

In [ ]:
# 初始化
Map = geemap.Map()

# 导入GEE数据
dem = ee.Image('USGS/SRTMGL1_003')  # SRTM高程数据
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')  
states = ee.FeatureCollection("TIGER/2018/States")

# 设置显示参数
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# 利用addLayer添加图层
Map.addLayer(dem, vis_params, 'STRM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(states, {}, "US States")

Map

## 2.2  draw features

In [ ]:
# 获取添加的特征列表
Map.draw_features

In [ ]:
# 获取添加的最后一个特征
Map.draw_last_feature

In [ ]:
# 将所有特征合并到一个集合中，获取其在states中的边界
roi = ee.FeatureCollection(Map.draw_features)
selected_states = states.filterBounds(roi)
Map.addLayer(selected_states, {}, "Selected states")

In [ ]:
# 裁剪所需要的区域
clipped_dem = dem.clip(selected_states)
Map.addLayer(clipped_dem, vis_params, 'Clipped image')

## 2.3  split_map

In [ ]:
# https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD
# 定义一个函数，返回GEE数据集包含的所有子集ID
def Get_GEEid(para):
    collection = ee.ImageCollection(para)
    return collection.aggregate_array('system:id').getInfo()
    
USGS_ID = Get_GEEid("USGS/NLCD")
USGS_ID

In [ ]:
# 分别选取2001年和2016年的土地利用数据进行对比
nlcd_old = ee.Image(USGS_ID[1]).select('landcover')
nlcd_new = ee.Image(USGS_ID[-1]).select('landcover')

# 把GEE的图层转换为ipyleaflet TileLayer
left_layer = geemap.ee_tile_layer(nlcd_old, {}, 'NLCD old')
right_layer = geemap.ee_tile_layer(nlcd_new, {}, 'NLCD new')

Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

## 2.4  Plotting

In [ ]:
Map = geemap.Map()

landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003') \
    .select([0, 1, 2, 3, 4, 6])
landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(landsat7, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

GPM = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filter(ee.Filter.date('2016-01-01', '2016-06-01'));
GPM_vis = {
  'min': 0.0,
  'max': 1000.0,
  'gamma': 2.5,
}
Map.addLayer(GPM, GPM_vis, 'NASA/GPM_L3/IMERG_V06')

Map

In [ ]:
在图层右上角勾选【Use Plotting】
在图层中点击任意一点即可查看信息

# 3  文件类型

## 3.1  json类型

In [ ]:
file_path = os.path.abspath('../data/us-states.json')

if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/us-states.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_states = json.load(f)

json_layer = GeoJSON(data=json_states, name='US States JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

# 转换为GEE类型
ee_states = geojson_to_ee(json_states)
Map.addLayer(ee_states, {}, "US States EE")

# 也可以重新转换为geo类型
# json_states_2 = ee_to_geojson(ee_states)
# json_layer_2 = GeoJSON(data=json_states_2, name='US States EE JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
# Map.add_layer(json_layer_2)

## 3.2  shapefiles

In [ ]:
Map = geemap.Map()
# 加载shp文件
states_shp = '../data/us-states.shp'
states = geemap.shp_to_ee(states_shp)
Map.addLayer(states, {}, 'US States')
Map

In [ ]:
# 导出shp文件
geemap.ee_to_shp(states, filename='../data/countries_new.shp')
# 导出其他格式的文件（更改后缀名即可：csv\json\kml\kmz）
geemap.ee_export_vector(states, filename='../data/states.csv')

## 3.3  Image

In [ ]:
# 加载Image图像
image = ee.Image('LE7_TOA_5YEAR/1999_2003')
# 定义下载区域
roi = ee.Geometry.Polygon(
  [[[101.0, 35.0],
    [101.0, 34.0],
    [100.0, 34.0],
    [100.0, 35.0]]], None, False)
# 输出文件路径
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')
filename

In [ ]:
# 裁剪影像
image = image.clip(roi).unmask()
# 输出影像（最后一个选项为是否输出每个波段为一张影像）
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)

## 3.4  ImageCollection

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
print(collection.aggregate_array('system:index').getInfo())
geemap.ee_export_image_collection(collection, out_dir=out_dir)

## 3.5  numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810') \
  .select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image. 
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

# 4  数据分析

## 4.1  聚类

In [ ]:
# 导入特征文件
file_path = os.path.abspath('../data/us-cities.json')
# 如果没有该文件则请求下载此文件
if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/us-cities.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_data = json.load(f)
    
json_data['features'][:4]

In [ ]:
Map = geemap.Map()

maker_cluster = MarkerCluster(
    markers=[Marker(location=feature['geometry']['coordinates'][::-1]) for feature in json_data['features']],
    name = 'Markers')

Map.add_layer(maker_cluster)
Map